In [23]:
import pandas as pd

data = pd.read_csv("Data.csv")

print(data.shape)
data.head()

(325, 1876)


,ID,3992.6,3990.7,3988.8,3986.8,3984.9,3983.0,3981.1,3979.1,3977.2,...,Substance,Natural /Synthetic,Plastic/other,framework,Abbreviation,Source,Source ID,Form,Colour,Method
0,1,0.00095,0.00087,0.00080,0.00074,0.00071,0.00068,0.00063,0.00059,0.00055,...,1_2_polybutadiene,synthetic polymer,plastic,polybutadiene,1_2_polybutadiene,"Scientific Polymer Products, Inc.",688,beads,transparent,ATR
1,2,0.00100,0.00098,0.00094,0.00090,0.00083,0.00078,0.00073,0.00065,0.00060,...,acrylonitrile_butadiene_styrene,synthetic polymer,plastic,acrylonitrile butadiene styrene,ABS,RSH Polymere,NaN,foil,light brown,ATR
2,3,0.00078,0.00074,0.00073,0.00073,0.00072,0.00069,0.00066,0.00060,0.00052,...,acrylonitrile_butadiene_styrene,synthetic polymer,plastic,acrylonitrile butadiene styrene,ABS,RSH Polymere,NaN,foil,light brown,ATR
3,4,0.00031,0.00029,0.00028,0.00029,0.00030,0.00027,0.00025,0.00026,0.00026,...,fibre_acetate,synthetic polymer,modified cellulose,cellulose acetate,fibre_acetate,Faserinstitut Bremen,NaN,fibre,white,ATR
4,5,0.00031,0.00030,0.00027,0.00023,0.00022,0.00022,0.00019,0.00018,0.00016,...,acrylonitrile_butadiene,synthetic polymer,plastic,acrylonitrile butadiene,acrylonitrile_butadiene,"Scientific Polymer Products, Inc.",530,crumbs,yellow,ATR


In [24]:
import numpy as np

print("NaN values:", np.isnan(X).sum().sum())
print("Inf values:", np.isinf(X).sum().sum())
print("Any zero-variance columns:", (X.std() == 0).sum())

NaN values: 0
Inf values: 0
Any zero-variance columns: 1


The first step is to separate the FTIR data from the categorical variables collumns. 
What we need to identify the type of material will be the "Abbreviation" collumn. 
That is our label we are going to try to predict (Y) 
The rest of the collumns will be used for training (X)

In [25]:
start_col = data.columns.get_loc("Substance")  

#we do not care about the ID collumn 0 
spectral_cols = data.columns[1:start_col]  


label_col = "Abbreviation"


In [26]:
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)

In [27]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
counts = y.value_counts()
rare_classes = counts[counts == 1]
print(rare_classes)


Series([], Name: count, dtype: int64)


In [29]:
valid_classes = counts[counts > 1].index


df_filtered = data[data["Abbreviation"].isin(valid_classes)]


meta_start_col = df_filtered.columns.get_loc("Substance")
spectral_cols = df_filtered.columns[1:meta_start_col]

X = df_filtered[spectral_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
y = df_filtered["Abbreviation"]

In [30]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [38]:

def augment_with_noise(X, y, noise_factor=0.005, n_copies=2, random_state=42):
    np.random.seed(random_state)
    X = np.asarray(X)
    y = np.asarray(y)

    X_aug = [X]
    y_aug = [y]

    for _ in range(n_copies):
        noise = np.random.normal(0, noise_factor * np.std(X, axis=0), X.shape)
        X_noisy = X + noise
        X_aug.append(X_noisy)
        y_aug.append(y)

    X_aug = np.vstack(X_aug)
    y_aug = np.concatenate(y_aug)

    print(f"✅ Added {n_copies * len(y)} new samples (total {len(y_aug)})")
    return X_aug, y_aug

In [37]:
print("X_scaled shape:", X_scaled.shape)
print("y shape:", y.shape)

X_scaled shape: (150, 1865)
y shape: (150,)


In [39]:
X_aug, y_aug = augment_with_noise(X_scaled, y, noise_factor=0.005, n_copies=2)
print("Original samples:", len(y))
print("After augmentation:", len(y_aug))

✅ Added 300 new samples (total 450)
Original samples: 150
After augmentation: 450


In [40]:

X_train, X_test, y_train, y_test = train_test_split( X_aug, y_aug, test_size=0.2, random_state=42, stratify=y_aug)

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = RandomForestClassifier(n_estimators=300, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))


✅ Accuracy: 1.0
